In [51]:
#lee logs de ingestas de tableros de producción

In [52]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import parametros

In [53]:
#FECHA ACTUAL 

In [54]:

now = datetime.now()
#now = datetime.now() - timedelta(hours = 8)

ahora = str(now.strftime("%Y-%m-%dT%H:%M:%S"))


In [55]:
#conecta elasticsearch

In [56]:
ahora

'2021-12-14T15:45:03'

In [57]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

In [58]:
#Leyendo archivo log trafico

In [40]:
 try:
    #logread = pd.read_csv('../archivos/sonicwall-curado.log', sep='\t', encoding = 'utf-8')
    header_list = ["column1"]
    logread = pd.read_csv('../archivos/ETL-Trafico.log', header=None,names=header_list, encoding = 'utf-8')
    #logread = pd.read_csv('../logs/ETL-Trafico.log', header=None,names=header_list, encoding = 'utf-8')
    logread.fillna('', inplace=True)
   
 except:
    print("fecha:",ahora,"- Error leyendo archivo trafico.log")
    exit()

In [41]:
logread1 = logread["column1"].str.split(expand=True)

In [42]:

logread1 = logread["column1"].str.split(expand=True)
logread1.columns = ['fecha ', 'last_name', 'last_name2', 'last_name3', 'last_name4', 'last_name5', 'last_name6', 'last_name7',  'last_name8', 'last_name9', 'last_name10', 'last_name11', 'last_name12', 'last_name13']

logread = pd.concat([logread, logread1], axis=1)

In [43]:
logread["d_last_name"] = logread['last_name']+logread['last_name2']

In [44]:
logread["d_last_name2"] = logread['last_name11']+logread['last_name12']

In [45]:
logread.fillna('', inplace=True)

In [46]:
logread_last_ = logread.tail(5)

In [47]:
#list(logread1.columns)

In [48]:
indice = 'alarma_trafico_4'

In [49]:
use_these_keys = ['last_name', 'last_name2', 'last_name9', 'last_name11', '@timestamp']

In [50]:
use_these_keys = ['d_last_name', 'd_last_name2', 'last_name9', 'last_name13', '@timestamp' ]
def filterKeys(document):
   return {key: document[key] for key in use_these_keys }
timestamp = datetime.now()
logread_last_['@timestamp'] = timestamp.isoformat()



def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": index,
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(logread_last_))
print("fecha:",ahora,"- Insertados en monitor-trafico:", salida[0])




<ipython-input-50-d857e99e0330>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logread_last_['@timestamp'] = timestamp.isoformat()


fecha: 2021-12-14T07:10:40 - Insertados en monitor-trafico: 5
